# Notebook for Training Model
This notebook contains all of the commands, environment setup, and code to begin training the model.

# 1. Downloading and Building Object Detection

In [ ]:
!git clone https://github.com/tensorflow/models.git ../model/models

In [ ]:
%cd ../model/models/research

In [ ]:
!protoc object_detection/protos/*.proto --python_out=.

In [ ]:
!copy object_detection/packages/tf2/setup.py ./
!python -m pip install .
!python object_detection/builders/model_builder_tf2_test.py
%cd ../../../notebooks

# 2. Generating TensorFlow Records for Data
Generating TF Records is an important step in generating the model as a whole.
This component allows TF to parse and train the model and for the distribution
of the model in a cross-platform format. See `src/notebooks/Generate TF Record` 
notebook for more detail.

In [ ]:
train_csv_input = "../data/train_labels.csv"
test_csv_input = "../data/test_labels.csv"
train_output_path = "../model/train.record"
test_output_path = "../model/test.record"
image_dir = "../data/images"

In [ ]:
from __future__ import division
from __future__ import print_function
from __future__ import absolute_import

import os
import io
import pandas as pd

from tensorflow.python.framework.versions import VERSION
if VERSION >= "2.0.0a0":
    import tensorflow.compat.v1 as tf
else:
    import tensorflow as tf

from PIL import Image
from object_detection.utils import dataset_util
from collections import namedtuple, OrderedDict

def class_text_to_int(row_label):
    if row_label == 'military tank':
        return 1
    elif row_label == 'military aircraft':
        return 2
    elif row_label == 'military truck':
        return 3
    elif row_label == 'civilian aircraft':
        return 4
    elif row_label == 'civilian car':
        return 5
    elif row_label == 'military helicopter':
        return 6
    else:
        return None
    
def split(df, group):
    data = namedtuple('data', ['filename', 'object'])
    gb = df.groupby(group)

    return [data(filename, gb.get_group(x)) for filename, x in zip(gb.groups.keys(), gb.groups)]

def create_tf_record(group, path):
    with tf.gfile.GFile(os.path.join(path, '{}'.format(group.filename)), 'rb') as fid:
        encoded_jpg = fid.read()

    encoded_jpg_io = io.BytesIO(encoded_jpg)
    image = Image.open(encoded_jpg_io)
    width, height = image.size

    filename = group.filename.encode('utf8')
    image_format = b'jpg'
    xmins = []
    xmaxs = []
    ymins = []
    ymaxs = []
    classes_text = []
    classes = []

    # Add all of the objects to the arrays.
    for index, row in group.object.iterrows():
        xmins.append(row['xmin'] / width)
        xmaxs.append(row['xmax'] / width)
        ymins.append(row['ymin'] / height)
        ymaxs.append(row['ymax'] / height)
        classes_text.append(row['class'].encode('utf8'))
        classes.append(class_text_to_int(row['class']))
        
    tf_record = tf.train.Example(features=tf.train.Features(feature = {
        'image/height': dataset_util.int64_feature(height),
        'image/width': dataset_util.int64_feature(width),
        'image/filename': dataset_util.bytes_feature(filename),
        'image/source_id': dataset_util.bytes_feature(filename),
        'image/encoded': dataset_util.bytes_feature(encoded_jpg),
        'image/format': dataset_util.bytes_feature(image_format),
        'image/object/bbox/xmin': dataset_util.float_list_feature(xmins),
        'image/object/bbox/xmax': dataset_util.float_list_feature(xmaxs),
        'image/object/bbox/ymin': dataset_util.float_list_feature(ymins),
        'image/object/bbox/ymax': dataset_util.float_list_feature(ymaxs),
        'image/object/class/text': dataset_util.bytes_list_feature(classes_text),
        'image/object/class/label': dataset_util.int64_list_feature(classes)
    }))
    
    return tf_record

In [ ]:
def create_record(csv_input, output_path):
    global image_dir
    writer = tf.python_io.TFRecordWriter(output_path)
    path = os.path.join(os.getcwd(), image_dir)

    print("Reading CSV label file...")
    examples = pd.read_csv(csv_input)
    grouped = split(examples, 'filename')

    print("Beginning compilation...")

    for group in grouped:
        tf_record = create_tf_record(group, path)
        writer.write(tf_record.SerializeToString())

    writer.close()
    output_path = os.path.join(os.getcwd(), output_path)
    print(f"Successfully created the TFRecords: \n{output_path}")

create_record(train_csv_input, train_output_path)
create_record(test_csv_input, test_output_path)

# 3. Downloading Model and Configuration

In [ ]:
%mkdir ../model/content
%cd ../model/content

First, the model itelf must be downloaded. This model is specifically used for object detection and classification, though outhers are also available.

In [ ]:
!wget http://download.tensorflow.org/models/object_detection/classification/tf2/20200710/mobilenet_v2.tar.gz
!tar -xvf mobilenet_v2.tar.gz
!rm mobilenet_v2.tar.gz

Next, the configuration must be downloaded.

In [ ]:
!wget https://raw.githubusercontent.com/tensorflow/models/master/research/object_detection/configs/tf2/ssd_mobilenet_v2_320x320_coco17_tpu-8.config
!mv ssd_mobilenet_v2_320x320_coco17_tpu-8.config mobilenet_v2.config

TensorFlow has provided a simple [script](https://blog.tensorflow.org/2021/01/custom-object-detection-in-browser.html) automatically adjusting the configuration, employed below. These values can be tweaked and adjusted for more accurate/longer training times, or for expirementation.

In [ ]:
num_classes = 6
batch_size = 4
num_steps = 7500
num_eval_steps = 1000

train_record_path = "../train.record"
test_record_path = "../test.record"
model_dir = "../training"
labelmap_path = "labelmap.pbtxt"
pipeline_config_path = "mobilenet_v2.config"
fine_tune_checkpoint = "mobilenet_v2/mobilenet_v2.ckpt-1"

import re

with open(pipeline_config_path) as f:
    config = f.read()

with open(pipeline_config_path, 'w') as f:
    config = re.sub('label_map_path: ".*?"', 'label_map_path: "{}"'.format(labelmap_path), config)

    config = re.sub(
        'fine_tune_checkpoint: ".*?"', 
        'fine_tune_checkpoint: "{}"'.format(fine_tune_checkpoint), 
        config
    )
    
    config = re.sub(
        '(input_path: ".*?)(PATH_TO_BE_CONFIGURED/train)(.*?")',
        'input_path: "{}"'.format(test_record_path), config
    )
    
    config = re.sub(
        '(input_path: ".*?)(PATH_TO_BE_CONFIGURED/val)(.*?")',
        'input_path: "{}"'.format(test_record_path), config
    )
    
    config = re.sub(
        'num_classes: [0-9]+',
        'num_classes: {}'.format(6), config
    )
    
    config = re.sub(
        'batch_size: [0-9]+',
        'batch_size: {}'.format(batch_size, config
)    )
    
    config = re.sub(
        'num_steps: [0-9]+',
        'num_steps: {}'.format(num_steps), config
    )
    
    f.write(config)
    print('Finished writing configuration')

# 4. Start Training Model

In [ ]:
!pip3 uninstall numpy -y
!pip3 install numpy
%cd ..
!python3 models/research/object_detection/model_main_tf2.py \
    --pipeline_config_path=content/{pipeline_config_path} \
    --model_dir={model_dir} \
    --alsologtostderr \
    --num_train_steps={num_steps} \
    --sample_1_of_n_eval_examples=1 \
    --num_eval_steps={num_eval_steps}

In [3]:
cd ../model

/Users/wpach/Dropbox/School/USC/Fall 2022/CSCE-585/Project/src/model


In [6]:
!pwd
%reload_ext tensorboard
%ls training/train
%tensorboard --logdir=training/

/Users/wpach/Dropbox/School/USC/Fall 2022/CSCE-585/Project/src/model
events.out.tfevents.1663877095.Walters-MBP.attlocal.net.14463.0.v2
events.out.tfevents.1663877244.Walters-MBP.attlocal.net.14480.0.v2


In [ ]:
!kill 14618